# Gap Analysis for IDW Interpolation in Weekly and Monthly interval

* [1. Weekly Preprocess](#week)
* [2. Monthly Preprocess](#month)

In [1]:
import pyproj
import pandas as pd
import numpy as np
import rasterio
import glob
import warnings
# import arcpy
from pyproj import Proj, Transformer
warnings.filterwarnings('ignore')

In [2]:
gis_path = r'F:\SEACAR_WQ_2024/GIS_Data/'
dfDis = pd.read_csv(gis_path + 'OEAT_Discrete_WQ-2024-May-06.csv', low_memory=False)

## 1. Weekly Preprocess <a class="anchor" id="week"></a>
### Aggregate the discrete data in same location as one row, and compute the period.

In [3]:
area_ab = ["GTM","EB","CH","BB","BBS"]
period_type = [" 52 week"," Month"]
def select_data_period(df,area,period):
    sheet_name = str(area) + str(period)
    df_period_table = pd.read_excel(gis_path + "All_Waterbodies_Season_Month_Week_Definitions.xlsx",sheet_name=sheet_name)
    df_select_area = df[df["WbodyAcronym"]==str(area)]
    df_period_table['Start Date'] = pd.to_datetime(df_period_table['Start Date'])
    df_period_table['End Date']   = pd.to_datetime(df_period_table['End Date'])
    sub_dfs = []

    for index, row in df_period_table.iterrows():
        start_date = row['Start Date']
        end_date   = row['End Date']
        sub_df = df_select_area[(df_select_area['SampleDate'] >= start_date) & (df_select_area['SampleDate'] < end_date)]
        sub_df['Period'] = row["Week"]
        sub_dfs.append(sub_df)
        
    df_period = pd.concat(sub_dfs,ignore_index=True)
    return df_period

In [4]:
dfDis['SampleDate'] = pd.to_datetime(dfDis['SampleDate'], format='%Y-%m-%d %H:%M:%S.%f')

In [5]:
sel_week_temp = []
for each in area_ab:
    df_week_temp = select_data_period(dfDis,str(each)," 52 week")
    df_week_temp_group = df_week_temp.groupby(['WaterBody','ParameterName','ParameterUnits','Year',
                                          'Latitude_DD','Longitude_DD','WbodyAcronym',"Period"])["ResultValue"].agg("mean").reset_index()
    sel_week_temp.append(df_week_temp_group)
df_week_select_Mean = pd.concat(sel_week_temp,ignore_index=True)

In [6]:
df_week_select_Mean.shape

(15880, 9)

In [7]:
param_shortnames = {
    'Salinity': 'Sal_ppt',
    'Total Nitrogen': 'TN_mgl',
    'Dissolved Oxygen': 'DO_mgl',
    'Turbidity':'Turb_ntu',
    'Secchi Depth':'Secc_m',
    'Water Temperature':'T_c'
}

In [8]:
df_week_select_Mean["ParaAcronym"] = df_week_select_Mean["ParameterName"].map(param_shortnames)

In [9]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
#     print(f"Transformed coordinates: {x}, {y}")
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
df_week_select_Mean[['x', 'y']] = df_week_select_Mean.apply(transform_coordinates, axis=1)

In [10]:
df_week_select_Mean.head(10)

,WaterBody,ParameterName,ParameterUnits,Year,Latitude_DD,Longitude_DD,WbodyAcronym,Period,ResultValue,ParaAcronym,x,y
0,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,2,7.340,DO_mgl,669975.848287,626752.656623
1,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,6,6.010,DO_mgl,669975.848287,626752.656623
2,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,11,5.545,DO_mgl,669975.848287,626752.656623
3,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,16,4.920,DO_mgl,669975.848287,626752.656623
4,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,20,6.955,DO_mgl,669975.848287,626752.656623
5,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,25,5.695,DO_mgl,669975.848287,626752.656623
6,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,29,6.360,DO_mgl,669975.848287,626752.656623
7,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,34,6.550,DO_mgl,669975.848287,626752.656623
8,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,37,7.310,DO_mgl,669975.848287,626752.656623
9,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.664722,-81.218056,GTM,2,7.725,DO_mgl,668862.259531,631692.835328


## Compute the total existing tiff files

In [11]:
raster_file_list = glob.glob(gis_path+"raster_output/idw_week/*.tif")
raster_name_list = []
for each in raster_file_list:
    name = each.split("\\")[-1]
    raster_name_list.append(name)

## Get the point value on the corresponding tiff file

In [12]:
value_list = []
for index,row in df_week_select_Mean.iterrows():
    wb     = row["WbodyAcronym"]
    para   = row["ParaAcronym"]
    year   = str(row["Year"])
    period = str(row["Period"])
    name   = str(f"{wb}_{para}_IDW_{year}_{period}.tif")
    x_cor  = row["x"]
    y_cor  = row["y"]
    path   = str(gis_path+"/raster_output/idw_week/"+name)
    if name in raster_name_list:
        with rasterio.open(path) as src:
            row, col = src.index(x_cor, y_cor)
            value = src.read(1)[row, col]
            if value < -9999:
                value = ""
            else:
                value = value
            value_list.append(value)
    else:
        value = ""
        value_list.append(value)


In [13]:
df_week_select_Mean["ConValue"] = value_list
df_week_gap = df_week_select_Mean[df_week_select_Mean["ConValue"]!=""]
df_week_gap["Difference"] = df_week_gap["ConValue"] - df_week_gap["ResultValue"]

## Compute RMSE, ME, and the number of discrete data in each week

In [14]:
def rmse(group):
    return np.sqrt(((group - group.mean()) ** 2).mean())
def me(group):
    return (group - group.mean()).mean()
def mean(group):
    return group.mean()

In [15]:
df_week_gap_group = df_week_gap.groupby(['WaterBody','ParameterName','Year', 'Period','WbodyAcronym','ParaAcronym'])
count = df_week_gap_group.size()
rmse_values = df_week_gap_group['Difference'].apply(rmse)
me_values = df_week_gap_group['Difference'].apply(me)
mean_values = df_week_gap_group['Difference'].apply(mean)
df_week_gap_summary = pd.DataFrame({
    'UsedCount': count,
    'RMSE': rmse_values,
    'ME': me_values,
    'MeanDif':mean_values
}).reset_index()
df_week_gap_summary.head()

,WaterBody,ParameterName,Year,Period,WbodyAcronym,ParaAcronym,UsedCount,RMSE,ME,MeanDif
0,Biscayne Bay,Dissolved Oxygen,2022,1,BB,DO_mgl,26,0.796764,-2.476651e-16,0.918978
1,Biscayne Bay,Dissolved Oxygen,2022,2,BB,DO_mgl,1,0.000000,0.000000e+00,0.112544
2,Biscayne Bay,Dissolved Oxygen,2022,3,BB,DO_mgl,1,0.000000,0.000000e+00,-0.990383
3,Biscayne Bay,Dissolved Oxygen,2022,4,BB,DO_mgl,1,0.000000,0.000000e+00,-0.447840
4,Biscayne Bay,Dissolved Oxygen,2022,5,BB,DO_mgl,27,1.022529,6.579099e-17,1.086931


In [16]:
df_week_select_Mean_group = df_week_select_Mean.groupby(['WaterBody','ParameterName','Year', 'Period','WbodyAcronym','ParaAcronym'])
count = df_week_select_Mean_group.size()
df_week_mean_summary = pd.DataFrame({'AllCount': count}).reset_index()
df_week_mean_summary.head()

,WaterBody,ParameterName,Year,Period,WbodyAcronym,ParaAcronym,AllCount
0,Big Bend Seagrasses,Dissolved Oxygen,2021,1,BBS,DO_mgl,2
1,Big Bend Seagrasses,Dissolved Oxygen,2021,2,BBS,DO_mgl,5
2,Big Bend Seagrasses,Dissolved Oxygen,2021,3,BBS,DO_mgl,4
3,Big Bend Seagrasses,Dissolved Oxygen,2021,4,BBS,DO_mgl,1
4,Big Bend Seagrasses,Dissolved Oxygen,2021,5,BBS,DO_mgl,5


In [17]:
df_week_gap_merge = pd.merge(df_week_gap_summary,df_week_mean_summary,on=['WaterBody','ParameterName','Year', 'Period','WbodyAcronym','ParaAcronym'])

In [18]:
df_week_gap_merge

,WaterBody,ParameterName,Year,Period,WbodyAcronym,ParaAcronym,UsedCount,RMSE,ME,MeanDif,AllCount
0,Biscayne Bay,Dissolved Oxygen,2022,1,BB,DO_mgl,26,0.796764,-2.476651e-16,0.918978,43
1,Biscayne Bay,Dissolved Oxygen,2022,2,BB,DO_mgl,1,0.000000,0.000000e+00,0.112544,6
2,Biscayne Bay,Dissolved Oxygen,2022,3,BB,DO_mgl,1,0.000000,0.000000e+00,-0.990383,7
3,Biscayne Bay,Dissolved Oxygen,2022,4,BB,DO_mgl,1,0.000000,0.000000e+00,-0.447840,6
4,Biscayne Bay,Dissolved Oxygen,2022,5,BB,DO_mgl,27,1.022529,6.579099e-17,1.086931,42
...,...,...,...,...,...,...,...,...,...,...,...
506,Guana Tolomato Matanzas,Water Temperature,2016,36,GTM,T_c,8,0.561621,0.000000e+00,0.430373,10
507,Guana Tolomato Matanzas,Water Temperature,2016,37,GTM,T_c,5,0.499004,8.881784e-17,1.261683,5
508,Guana Tolomato Matanzas,Water Temperature,2017,48,GTM,T_c,8,0.221336,0.000000e+00,0.294465,10
509,Guana Tolomato Matanzas,Water Temperature,2017,49,GTM,T_c,2,0.444253,0.000000e+00,-1.051744,2


In [19]:
df_week_gap_merge.to_csv(gis_path+"dis_gap_week.csv")

In [20]:
df_week_select_Mean.to_csv(gis_path + "week_difference.csv")

## 2. Monthly Preprocess <a class="anchor" id="month"></a>

In [21]:
area_ab = ["GTM","EB","CH","BB","BBS"]
period_type = [" 52 week"," Month"]
def select_data_period1(df,area,period):
    sheet_name = str(area) + str(period)
    df_period_table = pd.read_excel(gis_path + "All_Waterbodies_Season_Month_Week_Definitions.xlsx",sheet_name=sheet_name)
    df_select_area = df[df["WbodyAcronym"]==str(area)]
    df_period_table['Start Date'] = pd.to_datetime(df_period_table['Start Date'])
    df_period_table['End Date']   = pd.to_datetime(df_period_table['End Date'])
    sub_dfs = []

    for index, row in df_period_table.iterrows():
        start_date = row['Start Date']
        end_date   = row['End Date']
        sub_df = df_select_area[(df_select_area['SampleDate'] >= start_date) & (df_select_area['SampleDate'] < end_date)]
        sub_df['Period'] = row["Month"]
        sub_dfs.append(sub_df)
        
    df_period = pd.concat(sub_dfs,ignore_index=True)
    return df_period

In [22]:
sel_month_temp = []
for each in area_ab:
    df_month_temp = select_data_period1(dfDis,str(each)," Month")
    df_month_temp_group = df_month_temp.groupby(['WaterBody','ParameterName','ParameterUnits',
                                          'Year','Latitude_DD','Longitude_DD','WbodyAcronym',"Period"])["ResultValue"].agg("mean").reset_index()
    sel_month_temp.append(df_month_temp_group)
df_month_select_Mean = pd.concat(sel_month_temp,ignore_index=True)

In [23]:
df_month_select_Mean["ParaAcronym"] = df_month_select_Mean["ParameterName"].map(param_shortnames)

In [24]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
df_month_select_Mean[['x', 'y']] = df_month_select_Mean.apply(transform_coordinates, axis=1)

In [25]:
df_month_select_Mean.head()

,WaterBody,ParameterName,ParameterUnits,Year,Latitude_DD,Longitude_DD,WbodyAcronym,Period,ResultValue,ParaAcronym,x,y
0,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,1,7.340,DO_mgl,669975.848287,626752.656623
1,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,2,6.010,DO_mgl,669975.848287,626752.656623
2,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,3,5.545,DO_mgl,669975.848287,626752.656623
3,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,4,4.920,DO_mgl,669975.848287,626752.656623
4,Guana Tolomato Matanzas,Dissolved Oxygen,mg/L,2016,29.620011,-81.207694,GTM,5,6.955,DO_mgl,669975.848287,626752.656623


In [26]:
raster_file_list_m = glob.glob(gis_path+"raster_output/idw_month/*.tif")
raster_name_list_m = []
for each in raster_file_list_m:
    name = each.split("\\")[-1]
    raster_name_list_m.append(name)

In [27]:
value_list_m = []
for index,row in df_month_select_Mean.iterrows():
    wb     = row["WbodyAcronym"]
    para   = row["ParaAcronym"]
    year   = str(row["Year"])
    period = str(row["Period"])
    name   = str(f"{wb}_{para}_IDW_{year}_{period}.tif")
    x_cor  = row["x"]
    y_cor  = row["y"]
    path   = str(gis_path+"/raster_output/idw_month/"+name)
    if name in raster_name_list_m:
        with rasterio.open(path) as src:
            x,y = (x_cor,y_cor)
            row, col = src.index(x, y)
            value = src.read(1)[row, col]
            if value < -9999:
                value = ""
            else:
                value = value
            value_list_m.append(value)
    else:
        value = ""
        value_list_m.append(value)

In [28]:
df_month_select_Mean["ConValue"] = value_list_m
df_month_gap = df_month_select_Mean[df_month_select_Mean["ConValue"]!=""]
df_month_gap["Difference"] = df_month_gap["ConValue"] - df_month_gap["ResultValue"]

In [29]:
df_month_gap_group = df_month_gap.groupby(['WaterBody','ParameterName','Year', 'Period','WbodyAcronym','ParaAcronym'])
count = df_month_gap_group.size()
rmse_values = df_month_gap_group['Difference'].apply(rmse)
me_values = df_month_gap_group['Difference'].apply(me)
mean_values = df_month_gap_group['Difference'].apply(mean)
df_month_gap_summary = pd.DataFrame({
    'UsedCount': count,
    'RMSE': rmse_values,
    'ME': me_values,
    'MeanDif':mean_values
}).reset_index()
df_month_gap_summary.head()

,WaterBody,ParameterName,Year,Period,WbodyAcronym,ParaAcronym,UsedCount,RMSE,ME,MeanDif
0,Biscayne Bay,Dissolved Oxygen,2022,1,BB,DO_mgl,28,1.152005,-4.758099e-17,1.107885
1,Biscayne Bay,Dissolved Oxygen,2022,2,BB,DO_mgl,28,1.184647,-1.110223e-16,1.300088
2,Biscayne Bay,Dissolved Oxygen,2022,3,BB,DO_mgl,2,0.012193,0.000000e+00,-1.062569
3,Biscayne Bay,Dissolved Oxygen,2022,4,BB,DO_mgl,28,1.230934,1.586033e-17,0.424475
4,Biscayne Bay,Dissolved Oxygen,2022,5,BB,DO_mgl,34,1.277795,7.836868e-17,0.718093


In [30]:
df_month_select_Mean_group = df_month_select_Mean.groupby(['WaterBody','ParameterName','Year', 'Period','WbodyAcronym','ParaAcronym'])
count = df_month_select_Mean_group.size()
df_month_mean_summary = pd.DataFrame({'AllCount': count}).reset_index()
df_month_mean_summary.head()

,WaterBody,ParameterName,Year,Period,WbodyAcronym,ParaAcronym,AllCount
0,Big Bend Seagrasses,Dissolved Oxygen,2021,1,BBS,DO_mgl,12
1,Big Bend Seagrasses,Dissolved Oxygen,2021,2,BBS,DO_mgl,23
2,Big Bend Seagrasses,Dissolved Oxygen,2021,3,BBS,DO_mgl,7
3,Big Bend Seagrasses,Dissolved Oxygen,2021,4,BBS,DO_mgl,25
4,Big Bend Seagrasses,Dissolved Oxygen,2021,5,BBS,DO_mgl,27


In [31]:
df_month_gap_merge = pd.merge(df_month_gap_summary,df_month_mean_summary,on=['WaterBody','ParameterName','Year', 'Period','WbodyAcronym','ParaAcronym'])

In [32]:
df_month_gap_merge

,WaterBody,ParameterName,Year,Period,WbodyAcronym,ParaAcronym,UsedCount,RMSE,ME,MeanDif,AllCount
0,Biscayne Bay,Dissolved Oxygen,2022,1,BB,DO_mgl,28,1.152005,-4.758099e-17,1.107885,43
1,Biscayne Bay,Dissolved Oxygen,2022,2,BB,DO_mgl,28,1.184647,-1.110223e-16,1.300088,43
2,Biscayne Bay,Dissolved Oxygen,2022,3,BB,DO_mgl,2,0.012193,0.000000e+00,-1.062569,7
3,Biscayne Bay,Dissolved Oxygen,2022,4,BB,DO_mgl,28,1.230934,1.586033e-17,0.424475,44
4,Biscayne Bay,Dissolved Oxygen,2022,5,BB,DO_mgl,34,1.277795,7.836868e-17,0.718093,52
...,...,...,...,...,...,...,...,...,...,...,...
175,Guana Tolomato Matanzas,Water Temperature,2016,6,GTM,T_c,9,0.151723,-6.167906e-18,0.171766,10
176,Guana Tolomato Matanzas,Water Temperature,2016,7,GTM,T_c,15,0.544146,-5.921189e-17,2.509797,17
177,Guana Tolomato Matanzas,Water Temperature,2016,8,GTM,T_c,13,1.343141,5.124106e-17,-1.191999,15
178,Guana Tolomato Matanzas,Water Temperature,2016,9,GTM,T_c,13,0.746416,1.708035e-17,0.962894,15


In [33]:
df_month_gap_merge.to_csv(gis_path+"dis_gap_month.csv")

In [34]:
df_month_select_Mean.to_csv(gis_path + "month_difference.csv")